### ライブラリインポート

In [1]:
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, CSVLogger

import matplotlib.pyplot as plt
import os.path
import sys
import pprint
import numpy

#pprint.pprint(sys.path)

Using TensorFlow backend.


### 自前で用意したライブラリインポート ＋ データセットパス指定

In [2]:
from data import DataSet
from cnn_models import ResearchModels
import time

HOMEDIR = os.path.expanduser("~") + '/'
DATA_PATH = HOMEDIR + "dataset/UCF101/data_file.csv"
FRAME_DIR = HOMEDIR + "dataset/UCF101/UCF-101_frame"
TRAIN_DIR = FRAME_DIR + "/train"
TEST_DIR = FRAME_DIR + "/test"

data = DataSet(data_path=DATA_PATH)

### GPU制限(念の為)

In [3]:
import trainSetting 

trainSetting.GPU_Limit(0.5)
#trainSetting.GPU_LimitAllow()

### 学習を行う関数(参考コードそのまま)

In [4]:
def train(data_type, seq_length, model, saved_model=None,
          class_limit=None, image_shape=None,
          load_to_memory=False, batch_size=32, nb_epoch=100):
    # Helper: Save the model.
    checkpointer = ModelCheckpoint(
        filepath=os.path.join('data', 'checkpoints', model + '-' + data_type + \
            '.{epoch:03d}-{val_loss:.3f}.hdf5'),
        verbose=1,
        save_best_only=True)

    # Helper: TensorBoard
    tb = TensorBoard(log_dir=os.path.join('data', 'logs', model))

    # Helper: Stop when we stop learning.
    early_stopper = EarlyStopping(patience=5)

    # Helper: Save results.
    timestamp = time.time()
    csv_logger = CSVLogger(os.path.join('data', 'logs', model + '-' + 'training-' + \
        str(timestamp) + '.log'))

    # Get the data and process it.
    if image_shape is None:
        data = DataSet(
            seq_length=seq_length,
            class_limit=class_limit,
            data_path=DATA_PATH
        )
    else:
        data = DataSet(
            seq_length=seq_length,
            class_limit=class_limit,
            image_shape=image_shape,
            data_path=DATA_PATH
        )

    # Get samples per epoch.
    # Multiply by 0.7 to attempt to guess how much of data.data is the train set.
    steps_per_epoch = (len(data.data) * 0.7) // batch_size

    if load_to_memory:
        # Get data.
        X, y = data.get_all_sequences_in_memory('train', data_type, FRAME_DIR)
        X_test, y_test = data.get_all_sequences_in_memory('test', data_type, FRAME_DIR)
    else:
        # Get generators.
        generator = data.frame_generator(batch_size, 'train', data_type, FRAME_DIR)
        val_generator = data.frame_generator(batch_size, 'test', data_type, FRAME_DIR)

    # Get the model.
    rm = ResearchModels(len(data.classes), model, seq_length, saved_model)

    # Fit!
    if load_to_memory:
        # Use standard fit.
        rm.model.fit(
            X,
            y,
            batch_size=batch_size,
            validation_data=(X_test, y_test),
            verbose=1,
            callbacks=[tb, early_stopper, csv_logger],
            epochs=nb_epoch)
    else:
        # Use fit generator.
        rm.model.fit_generator(
            generator=generator,
            steps_per_epoch=steps_per_epoch,
            epochs=nb_epoch,
            verbose=1,
            callbacks=[tb, early_stopper, csv_logger, checkpointer],
            validation_data=val_generator,
            validation_steps=40,
            workers=4)

### 各種パラメータ設定

In [5]:
def trainStart(model='conv_3d', saved_model=None,class_limit=None
               ,seq_length=40,load_to_memory=False,batch_size=24,nb_epoch=1):
    
    # Chose images or features and image shape based on network.
    if model in ['conv_3d', 'c3d', 'lrcn']:
        data_type = 'images'
        image_shape = (80, 80, 3)
    elif model in ['lstm', 'mlp']:
        data_type = 'features'
        image_shape = None
    else:
        raise ValueError("Invalid model. See train.py for options.")
    
    train(data_type, seq_length, model, saved_model=saved_model,
          class_limit=class_limit, image_shape=image_shape,
          load_to_memory=load_to_memory, batch_size=batch_size, nb_epoch=nb_epoch)

### 学習！！

In [6]:
trainStart(model='mlp',nb_epoch=100)

Loading simple MLP.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 81920)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               41943552  
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
d

KeyboardInterrupt: 

In [ ]:
trainStart(model='lstm',nb_epoch=100)

In [ ]:
trainStart(model='conv_3d',nb_epoch=100)

In [ ]:
trainStart(model='c3d',nb_epoch=100)

In [ ]:
trainStart(model='lrcn',nb_epoch=100)